### 크롤링

In [1]:
from bs4 import BeautifulSoup
import requests
import json
from tqdm import tqdm

**발급 가능 카드 페이지 번호 추출**

In [ ]:
# 신용카드, 체크카드별 카드 개수
card_category = {'CRD': 1005,
                 'CHK': 429}
headers = {"User-Agent": "Mozilla/5.0"}

# 카드 종류(key), 카드 개수(value)로 돌리기
for cat,num in card_category.items():
    # 신용카드인 경우, 페이지 번호 추출
    if cat == 'CRD':
        url = f'https://api.card-gorilla.com:8080/v1/cards?p=1&perPage={num}&cate={cat}'
        response = requests.get(url, headers=headers)
        crd_num_json = response.json()
        
        # 발급 중단 카드 제거(is_discon)
        crd_num_list = [int(i.get('cid')) for i in tqdm(crd_num_json.get('data')) if i.get('is_discon')==False]

        print(f'신용카드 {len(crd_num_list)}개 완료')
    
    # 체크카드인 경우, 페이지 번호 추출
    elif cat == 'CHK':
        url = f'https://api.card-gorilla.com:8080/v1/cards?p=1&perPage={num}&cate={cat}'
        response = requests.get(url, headers=headers)
        chk_num_json = response.json()

        # 발급 중단 카드 제거(is_discon)
        chk_num_list = [int(i.get('cid')) for i in tqdm(chk_num_json.get('data')) if i.get('is_discon')==False]

        print(f'체크카드 {len(chk_num_list)}개 완료')

# 전체 카드 번호 통합 리스트 생성
page_list = crd_num_list+chk_num_list

In [ ]:
# 전체 카드 정보 넣을 딕셔너리
all_card = {}

# 카드 번호 크롤링 시작
for page_num in tqdm(page_list):
    response = requests.get(f'https://api.card-gorilla.`com:8080/v1/cards/{page_num}')

    # 페이지가 존재하지 않는 경우 넘기기
    if response.status_code !=200:
        continue
    
    # 크롤링 데이터 저장
    json_soup = response.json()
    
    # 카드 이름
    card_name = json_soup.get('name')
    all_card[card_name] = []

    # 카드사 이름
    all_card[card_name].append({'카드사' : json_soup.get('corp').get('name')})

    # 신용 체크 구분
    all_card[card_name].append({'종류' :  '신용카드' if json_soup.get('cate')=='CRD' else '체크카드'})

    # 해외 결제 여부
    forreign_val = json_soup.get('brand')
    all_card[card_name].append({'해외' : [ forreign_val[num].get('name') for num in range(len(forreign_val))]})

    # 주요 혜택 저장
    benefit_dict = {}
    for num in range(len(json_soup.get('key_benefit'))):
        benefit_temp = json_soup.get('key_benefit')[num]
        benefit_content = benefit_temp.get('comment')
        benefit_dict[benefit_temp.get('title')] = BeautifulSoup(benefit_content, 'html.parser').get_text(separator=' ', strip=True)

    all_card[card_name].append({'혜택' : benefit_dict})

    # 프로모션이 있는 경우 저장
    if json_soup.get('pr_container'):
        all_card[card_name].append({'프로모션' : BeautifulSoup(json_soup.get('pr_container'), 'html.parser').get_text(separator=' ', strip=True)})
    else:
        all_card[card_name].append({'프로모션' : '없음'})

    # 연회비, 연회비 상세 추출(html이 아닌 경우 무시하고 진행됨)
    if json_soup.get('annual_fee_basic') and json_soup.get('annual_fee_detail'):
        all_card[card_name].append({'연회비': json_soup.get('annual_fee_basic'),
                                                '연회비 상세': BeautifulSoup(json_soup.get('annual_fee_detail'), 'html.parser').get_text(separator=' ', strip=True)})

In [ ]:
with open('jinwoo_card.json', 'w', encoding='utf-8') as f:
    json.dump(all_card, f, ensure_ascii=False, indent=4)

In [ ]:
with open('jinwoo_card.json', 'r', encoding='utf-8') as f:
    all_card = json.load(f)

### json 파일 문자열로 변경

In [4]:
flatten_card_info = ''

for num in tqdm(range(len(all_card))):
    card_name = list(all_card.keys())[num]

    info = ' | '.join([f'{name} : {benefit}' for card in all_card[card_name] for  name, benefit in card.items()])

    # 청크 분류를 위해 기준점 생성
    flatten_card_info += '<카드정보 시작>'+ card_name + info + '<카드정보 끝>'

100%|██████████| 1106/1106 [00:00<00:00, 25544.75it/s]


### Chunking

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma 
from langchain_openai import OpenAIEmbeddings 
from langchain.schema import Document
from getpass import getpass 

In [7]:
MY_API_KEY = getpass("OpenAI API Key:")

In [31]:
# 혜택을 저장해보면 대부분 500이하, 문맥을 읽을 필요는 없어보였음
chunk_size = 350    # 하나의 청크의 글자 수(200~1500) 일반적인 문서에서는 작게, 기술문서는 크게
chunk_overlap = 0   # 인접 텍스트에서 중복으로 포함될 문자 수

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap = chunk_overlap
    ,separators=['<카드정보 시작>']
)

texts = r_splitter.split_text(flatten_card_info)

# 도큐먼트 형태로 변경해줘야 함
my_chunk = [Document(page_content=text) for text in texts]

In [ ]:
# 전체 카드개수 1106개 청킹
len(my_chunk)

1106

Embedding

In [35]:
# 임베딩 모델 호출
my_embedding = OpenAIEmbeddings(model="text-embedding-3-large",
                                api_key=MY_API_KEY
                               )

VectorDB

In [ ]:
my_directory = 'VectorStores/'
vectordb = Chroma.from_documents(documents = my_chunk,
                                 embedding = my_embedding,
                                 persist_directory = my_directory
                                )

In [42]:
vectordb._collection.count()
# vectordb.delete_collection()

ValueError: Chroma collection not initialized. Use `reset_collection` to re-create and initialize the collection. 

### 모델 생성

**프롬프트 엔지니어링**

In [ ]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 답변 예시 
examples = [
{"context": """  
    대학생  
    성별 : 여성
    나이 : 22세  
    직업: 대학생 (아르바이트 소득 월 80만 원)  
    소비 패턴: 카페, 편의점, 온라인 쇼핑, 대중교통  
    카드 사용 목적: 용돈 관리, 소소한 할인 혜택  
    주요 관심사: 편의점 및 카페 할인, 교통비 할인, OTT 구독 할인  
  """,  
  "output": """  
    삼성카드 taptap O 카드를 추천합니다.  
    이 카드는 대중교통 및 택시 이용 시 10% 결제일 할인을 제공하여  
    자주 사용하는 교통비 절감에 도움이 됩니다.  
    또한, 이동통신 요금 10% 할인 혜택이 있어  
    매달 정기적으로 나가는 고정비를 줄일 수 있습니다.  
    대학생들이 자주 이용하는 CGV 및 롯데시네마에서 5,000원 할인을 제공하며,  
    해외 결제 시 1.3% 적립 혜택이 있어 해외 직구 시에도 유리합니다.  
    따라서 생활비 절감 및 소소한 혜택을 원하는 대학생에게 적합한 카드입니다.  
  """
}
,

{"context": """  
    대기업 직장인  
    성별 : 남성
    나이 : 32세 
    직업: 대기업 연구원 (연봉 7,500만 원)  
    소비 패턴: 자동차 유지비, 해외 직구, 식당 및 카페  
    카드 사용 목적: 합리적인 소비, 생활비 최적화  
    주요 관심사: 자동차 관련 혜택, 해외 직구 할인, 고급 레스토랑 할인  
  """,  
  "output": """  
    KB국민 다담카드를 추천합니다.  
    이 카드는 SK주유소에서 리터당 60원 청구할인을 제공하여  
    차량 유지비 절감에 효과적입니다.  
    또한, KB투어 할인 서비스와 해외겸용 카드 옵션을 지원하여  
    해외 직구 및 여행 시에도 유용합니다.  
    대중교통 10% 청구할인과 통신비 10% 할인 혜택을 제공하여  
    생활비 절약에도 도움이 됩니다.  
    추가적으로, 영화 및 테마파크 할인 혜택도 있어  
    여가생활을 즐기는 데에도 적합한 카드입니다.  
  """
},


{"context": """  
    금융업 종사자  
    성별 : 남성
    나이 : 38세 
    직업: 투자 은행 펀드 매니저 (연봉 1억 2천만 원)  
    소비 패턴: 해외 출장, 고급 호텔, 프리미엄 서비스  
    카드 사용 목적: 프리미엄 라이프스타일 유지, 여행 혜택 극대화  
    주요 관심사: 공항 라운지, 호텔 VIP 혜택, 항공 마일리지  
  """,  
  "output": """  
    우리카드 로얄블루1000카드를 추천합니다.  
    이 카드는 연회비 100만 원의 프리미엄 카드로, 고급 라이프스타일을 즐기는 고객에게 적합합니다.  
    100만 원 상당의 선택형 기프트 바우처를 연 1회 제공하여 맞춤형 혜택을 누릴 수 있습니다.  
    공항 라운지 이용 및 항공 관련 서비스가 포함되어 있어 해외 출장이 잦은 분들에게 유용합니다.  
    컨시어지 서비스(Concierge Service)를 통해 여행, 쇼핑, 골프, 주유 등 다양한 프리미엄 혜택을 제공합니다.  
    호텔 및 여행 관련 프리미엄 서비스도 포함되어 있어 VIP 고객에게 최적화된 카드입니다.  
  """
}
]

# 예제 프롬프트 템플릿 정의
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "해당 페르소나에게 적절한 카드를 추천해주세요. {context}"),
        ("ai", "{output}"),
    ]
)

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    
    [
# 1. 당신은 대한민국 최고의 신용카드 전략기획부 팀장입니다. 당신은 제공받은 페르소나에게 적절한 카드를 vectordb에서 확인해서 추천해주어야 합니다.
# 2. 한국어로 답변을 해줘야 합니다.
# 3. few_shot_prompt는 context에 기재된 페르소나를 기준으로 output과 같은 답변을 해달라는 예시내용입니다. 실제 output값에 few_shot_prompt내용을 포함하지 마세요.
# 4. 답변은 자세할수록 좋아요, 모든 답변에 대해서 동일한 형식의 답변을 해주세요.
# 5. 카드 추천은 최대 2개까지만 해주세요.
        
        ("system", f"""
You are the team leader of the strategic planning department at South Korea's top credit card company. Your task is to recommend the most suitable credit card for a given persona by referencing the vector database (vectordb).
Please respond in Korean.
The few_shot_prompt provides examples of how you should format your responses based on the persona in the context. However, do not include the content of the few_shot_prompt in the actual output.
Your responses should be as detailed as possible. 
Always maintain a consistent format for all recommendations.
Please recommend two cards.
Please focus on your main interests.
         """),
         few_shot_prompt,
        ("human", "{context}"),
        
    ]
)

**예측할 페르소나 작성**

In [16]:
personas = [
   {"이준호": """ 
    자영업자
    성별 : 남성
    나이 : 45세 
    직업: 카페 운영 (연간 소득 1억 원)  
    소비 패턴: 비즈니스 경비 지출, 카드 결제 비중 높음, 차량 유지비  
    카드 사용 목적: 사업 운영 비용 절감, 높은 적립률, 세금 납부 혜택  
    주요 관심사: 사업 경비 적립, 주유 할인, 통신비 할인  
    """},

    {"박지연": """ 
    대학생
    성별 : 여성
    나이: 22세  
    직업: 대학생 (아르바이트 소득 월 80만 원)  
    소비 패턴: 카페, 편의점, 온라인 쇼핑, 대중교통  
    카드 사용 목적: 용돈 관리, 소소한 할인 혜택  
    주요 관심사: 편의점 및 카페 할인, 교통비 할인, OTT 구독 할인  
    """},

    {"최형민": """ 
    대기업 직장인
    성별 : 남성
    나이 : 32세
    직업: 대기업 연구원 (연봉 7,500만 원)  
    소비 패턴: 자동차 유지비, 해외 직구, 식당 및 카페  
    카드 사용 목적: 합리적인 소비, 생활비 최적화  
    주요 관심사: 자동차 관련 혜택, 해외 직구 할인, 고급 레스토랑 할인  
    """},

    {"박상현": """ 
    금융업 종사자
    성별: 남성
    나이: 38세 
    직업: 투자 은행 펀드 매니저 (연봉 1억 2천만 원)  
    소비 패턴: 해외 출장, 고급 호텔, 프리미엄 서비스  
    카드 사용 목적: 프리미엄 라이프스타일 유지, 여행 혜택 극대화  
    주요 관심사: 공항 라운지, 호텔 VIP 혜택, 항공 마일리지  
    """},

    {"윤서현": """ 
    주부
    성별: 여성
    나이:42세 
    직업: 전업주부 (가계 월 소득 900만 원)  
    소비 패턴: 대형 마트, 백화점, 가족 외식, 자녀 교육비  
    카드 사용 목적: 가계 경제 관리, 생활비 절감  
    주요 관심사: 마트 및 백화점 할인, 교육비 할인, 문화 생활 혜택  
    """}
]

**R_QA1 모델**

In [ ]:
# !pip install -U langchain-openai

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

chat_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key=MY_API_KEY
)

# 검색기 생성
my_retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# few_shot_prompt와 gpt3.5를 결합 (few_shot을 적용하기 위해서는 create_stuff_documents_chain필요)
combine_docs_chain = create_stuff_documents_chain(chat_model, 
                                                  final_prompt
                                                  )

# 검색기와 모델+프롬프트 체인 생성
retrieval_chain = create_retrieval_chain(my_retriever, 
                                         combine_docs_chain)

for human in personas:
    answer = retrieval_chain.invoke({"input": f"해당 페르소나에 맞는 카드를 추천해주세요.{list(human.values())[0]}"})
    print(f'{list(human.keys())[0]}씨를 위한 카드 추천')
    print(answer.get('input'))
    print('추천 카드')
    print(answer.get('answer'))
    print('-'*100)

이준호씨를 위한 카드 추천
해당 페르소나에 맞는 카드를 추천해주세요. 
    자영업자
    성별 : 남성
    나이 : 45세 
    직업: 카페 운영 (연간 소득 1억 원)  
    소비 패턴: 비즈니스 경비 지출, 카드 결제 비중 높음, 차량 유지비  
    카드 사용 목적: 사업 운영 비용 절감, 높은 적립률, 세금 납부 혜택  
    주요 관심사: 사업 경비 적립, 주유 할인, 통신비 할인  
    
추천 카드
첫 번째 카드로는 "토스유스카드"를 추천합니다.  
이 카드는 토스에서 제공하는 체크카드로, 국내에서의 온라인 및 간편결제에 특화되어 있습니다. 교통카드로도 사용 가능하여 대중교통을 이용하는 데 편리합니다. 연회비는 국내전용이며 발급 수수료가 없는 점이 장점입니다.  
이 카드는 주요 관심사인 간편결제 및 교통비 할인에 적합할 것으로 판단됩니다.  

두 번째 카드로는 "프레딧 하나카드"를 추천합니다.  
해외에서도 사용 가능한 VISA 신용카드로, 다양한 혜택을 제공합니다. 온라인 쇼핑, 카페/디저트, 간편결제, 통신 등 다양한 분야에서 할인 혜택을 받을 수 있습니다. 연회비는 국내전용과 해외겸용 모두 12,000원이며, 연간 이용금액에 따라 연회비 면제 혜택이 있는 점도 고려해 볼 만합니다.  
해외 직구 및 프리미엄 서비스에 관심이 있는 분들에게 적합한 카드입니다.
----------------------------------------------------------------------------------------------------
박지연씨를 위한 카드 추천
해당 페르소나에 맞는 카드를 추천해주세요. 
    대학생
    성별 : 여성
    나이: 22세  
    직업: 대학생 (아르바이트 소득 월 80만 원)  
    소비 패턴: 카페, 편의점, 온라인 쇼핑, 대중교통  
    카드 사용 목적: 용돈 관리, 소소한 할인 혜택  
    주요 관심사: 편의점 및 카페 할인, 교통비 할인, OTT 구독 할인 

In [18]:
from langchain_cohere import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
from openai import OpenAI
COHERE_API_KEY = getpass("Cohere API KEY :")

In [22]:
# 코히어 리랭크
my_rerank = CohereRerank(cohere_api_key = COHERE_API_KEY,
                         model = "rerank-v3.5" # 
                        )

# mmr 추가
mmr_retriever = vectordb.as_retriever(search_type ="mmr",
                                      search_kwargs={"fetch_k":150, # 백터DB에서 유사도 높은 문서 150개 추출
                                                     "k":3,         # 150개중 반환할 문서 3개
                                                     "lambda_mult" : 0.7} # 다양성보다는 관련성에 초점을 두고 1에 가깝게 설정
                                         )
# 코히어 + mmr 검색기 생성
compression_retriever = ContextualCompressionRetriever(base_compressor = my_rerank,
                                                       base_retriever = mmr_retriever
                                                      )

retrieval_rerank_chain = create_retrieval_chain(compression_retriever, 
                                         combine_docs_chain)


client= OpenAI(api_key=MY_API_KEY)


for human in personas:
    # 결과값 비교할 gpt-3.5-turbo 모델
    answer_35 = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                            messages = [{'role' : 'user',
                                                         'content' : f'아래 정보에 고객에게 맞는 카드를 추천해주세요. {list(human.values())[0]}'}],
                                            temperature = 0
                                           )
    # Rerank + mmr RAG모델
    answer_rag = retrieval_rerank_chain.invoke({"input": f"해당 페르소나에 맞는 카드를 추천해주세요.{list(human.values())[0]}"})
    print()
    print(f'<{list(human.keys())[0]}씨를 위한 카드 추천>')
    print(answer_rag.get('input'))
    print()
    print('[GPT-3.5-turbo모델 추천]')
    print(answer_35.choices[0].message.content)
    print('-'*100)
    print()
    print('[RAG모델 추천]')
    print(answer_rag.get('answer'))
    print('='*100)


<이준호씨를 위한 카드 추천>
해당 페르소나에 맞는 카드를 추천해주세요. 
    자영업자
    성별 : 남성
    나이 : 45세 
    직업: 카페 운영 (연간 소득 1억 원)  
    소비 패턴: 비즈니스 경비 지출, 카드 결제 비중 높음, 차량 유지비  
    카드 사용 목적: 사업 운영 비용 절감, 높은 적립률, 세금 납부 혜택  
    주요 관심사: 사업 경비 적립, 주유 할인, 통신비 할인  
    

[GPT-3.5-turbo모델 추천]
해당 고객에게는 비즈니스 신용카드를 추천드립니다. 비즈니스 신용카드는 사업 운영 비용을 절감하고 높은 적립률을 제공해주며, 세금 납부 혜택도 받을 수 있습니다. 또한 주유 할인이나 통신비 할인 혜택을 제공하는 카드를 선택하시면 소비 패턴에 맞게 혜택을 누릴 수 있을 것입니다. 이외에도 카드사마다 다양한 혜택을 제공하니, 고객님의 운영하는 카페와 관련된 혜택도 확인해보시면 좋을 것 같습니다.
----------------------------------------------------------------------------------------------------

[RAG모델 추천]
위의 정보를 참고하여 다음의 두 가지 카드를 대기업 직장인에게 추천합니다.

1. 삼성카드 개인사업자 신용카드
   - 해외 결제: Mastercard
   - 혜택: 사업장 운영 경비 1.5% 결제일 할인, 온라인 간편결제/커피전문점/해외 1.5% 결제일 할인, 국내 가맹점 1% 결제일 할인, 세무지원 서비스
   - 연회비: 국내전용 15,000원 / 해외겸용 15,000원
   - 특징: 사업 운영에 필요한 경비를 할인 받을 수 있으며, 해외 결제 시에도 혜택을 누릴 수 있는 카드입니다.

2. 현대카드 MY BUSINESS ZERO Food&Drink 신용카드
   - 해외 결제: VISA, Mastercard
   - 혜택: 사업성 경비 영역에서 이용 시 1.5% 할인, 모든 가맹점에서 이용 시 0.7% 